In [28]:
import cimgraph.data_profile.cimhub_2023 as cim
from uuid import UUID
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models import DistributedArea
import cimgraph.queries.sparql as sparql
import cimgraph.utils as utils
from defusedxml.ElementTree import parse
cim_profile = 'cimhub_2023'

In [31]:
from cimgraph.databases.fileparsers import XMLFile

# params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
# params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee123_pv.xml", cim_profile=cim_profile, iec61970_301=8)
# params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/IEEE118_CIM.xml", cim_profile=cim_profile, iec61970_301=8)
params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee9500bal.xml", cim_profile=cim_profile, iec61970_301=8)

xml_file = XMLFile(params)

In [30]:
%load_ext line_profiler

# def create_new_graph():
#     xml_file.create_new_graph({})

%lprun -f xml_file.create_new_graph xml_file.create_new_graph({})

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-09 s

Total time: 1.1238 s
File: /home/ande188/CIM-Graph/cimgraph/databases/fileparsers/xml_parser.py
Function: create_new_graph at line 50

Line #      Hits         Time  Per Hit   % Time  Line Contents
    50                                               def create_new_graph(self, container: object) -> dict[type, dict[UUID, object]]:
    51                                                   
    52         1 1123801494.0    1e+09    100.0          asyncio.run(self.create_new_graph_async())
    53         1        966.0    966.0      0.0          return self.graph

In [32]:
import cProfile

cProfile.run('xml_file.create_new_graph({})',sort=1)

         843800692 function calls in 365.731 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
102236099   92.072    0.000   92.072    0.000 encoder.py:204(iterencode)
102236099   88.904    0.000  319.377    0.000 cimhub_2023.py:63(__repr__)
102236099   51.867    0.000  157.964    0.000 encoder.py:182(encode)
102382612   48.610    0.000   48.610    0.000 uuid.py:279(__str__)
   146513   31.272    0.000  352.540    0.002 __init__.py:129(create_edge)
102236099   24.070    0.000  182.034    0.000 __init__.py:183(dumps)
205640349   10.050    0.000   10.050    0.000 {built-in method builtins.isinstance}
102236099    4.045    0.000    4.045    0.000 {method 'join' of 'str' objects}
   156342    2.142    0.000    2.311    0.000 {built-in method builtins.eval}
    69307    2.013    0.000  359.068    0.005 xml_parser.py:80(parse_edges)
   277233    1.257    0.000    1.496    0.000 base_events.py:772(_call_soon)
   639561    1.136    0

In [ ]:
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
# feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" #123 node
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=xml_file, container=feeder, distributed=False)

In [ ]:
# network.pprint(cim.ACLineSegment)

In [ ]:
graph = {}
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=7)
database = BlazegraphConnection(params)

In [ ]:
rdf = '''{http://www.w3.org/1999/02/22-rdf-syntax-ns#}'''
# Parse the XML file
tree = parse('./test_models/ieee13_assets.xml')
# tree = parse('./test_models/ieee9500bal.xml')
root = tree.getroot()

class_index = {}

# # Function to extract tag without namespace
# def extract_tag(tag):
#     return tag.split('}')[-1]

# Iterate over the elements and create dataclass instances
for element in root:

    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        class_index[identifier] = cim_class
        obj = database.create_object(graph, cim_class, uri)

for element in root:      
    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        
        obj = graph[cim_class][UUID(uri)]
        for sub_element in element:
            sub_tag = sub_element.tag.split('}')[-1]
            association = database.check_attribute(cim_class, sub_tag)
            try:
                edge_uri = sub_element.attrib[f'{rdf}resource'].split('uuid:')[-1]
            except:
                edge_uri = None

            if edge_uri is not None:
                if database.namespace not in edge_uri:
                    edge_uuid = UUID(edge_uri.strip('_').lower())                  
                    edge_class = class_index[edge_uuid]
                    database.create_edge(graph, cim_class, identifier, sub_tag, edge_class, edge_uri)
                    reverse = cim_class.__dataclass_fields__[association].metadata['inverse']
                    database.create_edge(graph, edge_class, edge_uuid, reverse, cim_class, graph[cim_class][identifier].uri())
                else:
                    enum_text = edge_uri.split(database.namespace)[1]
                    enum_text = enum_text.split('>')[0]
                    enum_class = enum_text.split('.')[0]
                    enum_value = enum_text.split('.')[1]
                    edge_enum = eval(f'cim.{enum_class}(enum_value)')
                    if association is not None:
                        setattr(graph[cim_class][identifier], association, edge_enum)
            else:
                if association is not None:
                    setattr(obj, association, sub_element.text)


In [ ]:
len(graph[cim.ACLineSegment].keys())

In [ ]:
enum_class